# Yelp API - Lab


## Introduction 

Now that we've seen how the Yelp API works and some basic Folium visualizations, it's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [1]:
#Your code here
import requests

In [2]:
api_key = 'N4EQR8ZJdicb_LFjUsgRbsKao08lXWLpa_VTtC4BqPOpEC4Uw5otYoAeb2eJhJwvhLn0uQIcqvSbRGL8EABX7lxhiY4mH-6tUKIaEbjdCdfHisQYNHueVZm4C9_eXHYx'
headers = {'Authorization': 'Bearer {}'.format(api_key)}
url = 'https://api.yelp.com/v3/businesses/search'
term = 'pizza'
location = 'Arlington VA'
SEARCH_LIMIT = 10
url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT
            }

In [3]:
resp = requests.get(url, headers=headers, params=url_params)

In [4]:
resp.json()['businesses']

[{'id': 'd82VK9Ve7hq1WROlsA4fIA',
  'alias': 'pupatella-arlington-3',
  'name': 'Pupatella',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/92Ert5m9_oJ1JtBpPRoRjw/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/pupatella-arlington-3?adjust_creative=OcDckSratin8g4BhRxu4_A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=OcDckSratin8g4BhRxu4_A',
  'review_count': 1784,
  'categories': [{'alias': 'pizza', 'title': 'Pizza'},
   {'alias': 'italian', 'title': 'Italian'},
   {'alias': 'gelato', 'title': 'Gelato'}],
  'rating': 4.0,
  'coordinates': {'latitude': 38.87765, 'longitude': -77.12223},
  'transactions': [],
  'price': '$$',
  'location': {'address1': '5104 Wilson Blvd',
   'address2': '',
   'address3': '',
   'city': 'Arlington',
   'zip_code': '22205',
   'country': 'US',
   'state': 'VA',
   'display_address': ['5104 Wilson Blvd', 'Arlington, VA 22205']},
  'phone': '+15713127230',
  'display_phone': '(571) 312-7230',
  'distance': 2

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [5]:
# Your code here; use a function or loop to retrieve all the results from your original request
import pandas as pd
import time

In [6]:
def yelp_call(url_params, api_key):
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    url = 'https://api.yelp.com/v3/businesses/search'
    resp = requests.get(url, headers=headers, params=url_params)
    df = pd.DataFrame(resp.json()['businesses'])
    return df

def all_results(url_params, api_key):
    num = resp.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    dfs = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        dfs.append(yelp_call(url_params, api_key))
        time.sleep(1)
        cur += 50
    df = pd.concat(dfs, ignore_index=True)
    return df
term = 'pizza'
location = 'Arlington VA'
SEARCH_LIMIT = 50
url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT
            }
df = all_results(url_params, api_key)
print(len(df))
df.head()

1200 total matches found.
1000


,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,pupatella-arlington-3,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 38.87765, 'longitude': -77.12223}",(571) 312-7230,2318.059492,d82VK9Ve7hq1WROlsA4fIA,https://s3-media4.fl.yelpcdn.com/bphoto/92Ert5...,False,"{'address1': '5104 Wilson Blvd', 'address2': '...",Pupatella,+15713127230,$$,4.0,1784,[],https://www.yelp.com/biz/pupatella-arlington-3...
1,and-pizza-arlington-3,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 38.87966, 'longitude': -77.10725}",(202) 558-7549,1102.282948,Uu6A4edyD6NUlqqAo2Cehw,https://s3-media1.fl.yelpcdn.com/bphoto/XQXi2b...,False,"{'address1': '3924 Wilson Blvd', 'address2': '...",&pizza,+12025587549,$$,4.0,92,"[delivery, pickup]",https://www.yelp.com/biz/and-pizza-arlington-3...
2,stone-hot-pizza-arlington,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 38.88614, 'longitude': -77.09738}",(571) 290-0171,1163.331935,szrZachMq32nl_RcAV_oXg,https://s3-media1.fl.yelpcdn.com/bphoto/aoL-FK...,False,"{'address1': '3217 Washington Blvd', 'address2...",Stone Hot Pizza,+15712900171,NaN,4.5,13,"[delivery, pickup]",https://www.yelp.com/biz/stone-hot-pizza-arlin...
3,extreme-pizza-arlington-arlington,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 38.86058, 'longitude': -77.05592}",(703) 271-1020,3823.361058,tncF1z3fOoOeYuGWDPO-Ig,https://s3-media4.fl.yelpcdn.com/bphoto/I-qCSr...,False,"{'address1': '1419 S Fern St', 'address2': '',...",Extreme Pizza - Arlington,+17032711020,$$,4.0,407,"[delivery, pickup]",https://www.yelp.com/biz/extreme-pizza-arlingt...
4,fire-works-pizza-arlington,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 38.889879, 'longitude': -77.087299}",(703) 527-8700,1721.114229,7cWSQZ3VLNnGf9UDWyupMA,https://s3-media2.fl.yelpcdn.com/bphoto/eAY2EB...,False,"{'address1': '2350 Clarendon Blvd', 'address2'...",Fire Works Pizza,+17035278700,$$,4.0,648,"[delivery, pickup]",https://www.yelp.com/biz/fire-works-pizza-arli...


## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [7]:
#Your code here
print(df.describe(), '\n')
print(df.price.value_counts(), '\n')
sum(df.coordinates.isna())


           distance       rating  review_count
count   1000.000000  1000.000000   1000.000000
mean    6035.464989     3.488500    308.544000
std     2527.762242     0.736315    623.801059
min      626.987197     1.000000      1.000000
25%     4473.991861     3.000000     46.000000
50%     6229.914906     3.500000    141.500000
75%     7684.894081     4.000000    358.250000
max    28822.196944     5.000000  12569.000000 

$$      548
$       312
$$$      46
$$$$      8
Name: price, dtype: int64 



0

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [8]:
#Your code here
#!pip install folium
import folium

In [9]:
list(resp.json()['region']['center'].values())

[-77.09552764892578, 38.87578821270115]

In [11]:
base_map = folium.Map([38.87578821270115, -77.09552764892578], zoom_start=13)


In [13]:
for i in range(len(df)):
    lat = list(df.iloc[i,2].values())[0]
    lon = list(df.iloc[i,2].values())[1]
    if type(lat)==type(None) or type(lon)==type(None):
        pass
    else:
        lat = list(df.iloc[i,2].values())[0]
        lon = list(df.iloc[i,2].values())[1]
        popup_text = "Name: {}, \n Price: {}".format(df.name.iloc[i], df.price.iloc[i])
        popup = folium.Popup(popup_text, parse_html=True)
        marker = folium.Marker(location=[lat, lon], popup=popup)
        marker.add_to(base_map)
base_map

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!